In [0]:
# -----------------------------------------------------------
# 1. Preparación: Lectura y procesamiento de la data original
# -----------------------------------------------------------
from pyspark.sql.functions import expr, col, min, mean, greatest,max
from databricks.feature_store import FeatureStoreClient
from mlflow.models import infer_signature

In [0]:
# Cargamos las tablas base
base_atributos = spark.table("databricks_clase.prueba_schema.base_atributos")
base_cliente = spark.table("databricks_clase.prueba_schema.base_cliente")
base_trx = spark.table("databricks_clase.prueba_schema.base_trx")

In [0]:
# Realizamos los joins para consolidar la información
join_1 = base_atributos.join(base_cliente, on=["periodo", "id_cliente"], how="left")
tabla_consolidada = join_1.join(base_trx, on=["periodo", "id_cliente"], how="left")

# Imputación de valores nulos en columnas categóricas
tabla_consolidada = tabla_consolidada.fillna({
    "tipo_producto": "Desconocido",
    "departamento": "Desconocido",
    "canal": "Desconocido"
})

# Imputación en columnas numéricas con la mediana
numeric_cols = ["monto_1m", "monto_2m", "monto_3m", "frecuencia_1m", "frecuencia_2m", "frecuencia_3m"]
for col_name in numeric_cols:
    median_value = tabla_consolidada.approxQuantile(col_name, [0.5], 0.01)[0]
    tabla_consolidada = tabla_consolidada.fillna({col_name: median_value})

# Creación de nuevas características
tabla_consolidada = tabla_consolidada.withColumn(
    "monto_total",
    expr("monto_1m + monto_2m + monto_3m + monto_4m + monto_5m + monto_6m")
)
tabla_consolidada = tabla_consolidada.withColumn(
    "tendencia_monto",
    expr("(monto_1m - monto_6m) / monto_6m")
)

# Otros ajustes en la data
tabla_consolidada = tabla_consolidada.drop('__index_level_0__')
tabla_consolidada = tabla_consolidada.filter(col("flg_churn").isNotNull())

columnas_cero = ["incidencias_a", "incidencias_b", "crossell", "ultima_compra_2m", "ultima_compra_3m"]
tabla_consolidada = tabla_consolidada.fillna({col: 0 for col in columnas_cero})

periodo_minimo = tabla_consolidada.select(min("periodo")).collect()[0][0]
tabla_consolidada = tabla_consolidada.fillna({"periodo_creacion": periodo_minimo})

columnas_menos_uno = ["segmento_pago", "segmento_cliente"]
tabla_consolidada = tabla_consolidada.fillna({col: -1 for col in columnas_menos_uno})

tasa_media = tabla_consolidada.select(mean("tasa")).collect()[0][0]
tabla_consolidada = tabla_consolidada.fillna({"tasa": tasa_media})

In [0]:
periodo_max =tabla_consolidada.agg(max("periodo")).collect()[0][0]
tabla_consolidada_mensual = tabla_consolidada.filter(tabla_consolidada.periodo == periodo_max)

In [0]:
spark.sql("DROP TABLE IF EXISTS databricks_clase.prueba_schema.base_consolidada_v2 PURGE")


DataFrame[]

In [0]:
%sql
CREATE TABLE databricks_clase.prueba_schema.base_consolidada_v2 (
    periodo BIGINT,
    id_cliente BIGINT,
    tiempo_permanencia BIGINT,
    flg_vip DOUBLE,
    incidencias_a DOUBLE NOT NULL,
    incidencias_b DOUBLE NOT NULL,
    tipo_producto STRING NOT NULL,
    periodo_creacion BIGINT NOT NULL,
    departamento STRING NOT NULL,
    segmento_pago BIGINT NOT NULL,
    canal STRING NOT NULL,
    segmento_cliente BIGINT NOT NULL,
    crossell DOUBLE NOT NULL,
    tasa DOUBLE NOT NULL,
    monto_1m DOUBLE NOT NULL,
    monto_2m DOUBLE NOT NULL,
    monto_3m DOUBLE NOT NULL,
    monto_4m DOUBLE,
    monto_5m DOUBLE,
    monto_6m DOUBLE,
    cantidad_1m DOUBLE,
    cantidad_2m DOUBLE,
    cantidad_3m DOUBLE,
    cantidad_6m DOUBLE,
    frecuencia_1m DOUBLE NOT NULL,
    frecuencia_2m DOUBLE NOT NULL,
    frecuencia_3m DOUBLE NOT NULL,
    ultima_compra_1m DOUBLE,
    ultima_compra_2m DOUBLE NOT NULL,
    ultima_compra_3m DOUBLE NOT NULL,
    flg_churn DOUBLE,
    monto_total DOUBLE,
    tendencia_monto DOUBLE
)
USING DELTA;


In [0]:
# Guardamos la tabla consolidada en Unity Catalog para persistencia
tabla_consolidada.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("databricks_clase.prueba_schema.base_consolidada_v2")

In [0]:
# -----------------------------------------------------------
# 2. Creación de dos Feature Store tables
# -----------------------------------------------------------
fs = FeatureStoreClient()

In [0]:
# --- 2.1. Tabla 1: Features mensuales (conjunto principal de features) ---
feature_columns_1 = [
    'periodo', 'id_cliente', 'tiempo_permanencia', 'flg_vip', 'incidencias_a', 'incidencias_b', 
    'tipo_producto', 'periodo_creacion', 'departamento', 'segmento_pago', 'canal', 
    'segmento_cliente', 'crossell', 'tasa', 'monto_1m', 'monto_2m', 'monto_3m', 'monto_4m', 
    'monto_5m', 'monto_6m', 'cantidad_1m', 'cantidad_2m', 'cantidad_3m', 'cantidad_6m', 
    'frecuencia_1m', 'frecuencia_2m', 'frecuencia_3m', 'ultima_compra_1m', 'ultima_compra_2m', 
    'ultima_compra_3m', 'monto_total', 'tendencia_monto'
]

# Suponemos que ya se ha generado la data mensual consolidada
df_mensual = spark.table("databricks_clase.prueba_schema.base_consolidada_v2")

feature_table_name_1 = "databricks_clase.prueba_schema.base_consolidada_mensual_feats_v2"

# Crear la tabla de features 1 en el Feature Store
fs.create_table(
    name=feature_table_name_1,
    primary_keys=["id_cliente"],
    schema=df_mensual.select(*feature_columns_1).schema,
    description="Feature Store con features mensuales consolidados (sin flg_churn)"
)

# Escribir la data en el Feature Store (excluimos 'id_cliente' de las columnas de features, pues es clave primaria)
columns_to_write_1 = [col for col in feature_columns_1 if col != "id_cliente"]
df_features = df_mensual.select("id_cliente", *columns_to_write_1).dropDuplicates(["id_cliente"])
fs.write_table(
    name=feature_table_name_1,
    df=df_features,
    mode="overwrite"
)

2025/02/26 02:11:31 INFO databricks.ml_features._compute_client._compute_client: Setting columns ['id_cliente'] of table 'databricks_clase.prueba_schema.base_consolidada_mensual_feats_v2' to NOT NULL.
2025/02/26 02:11:32 INFO databricks.ml_features._compute_client._compute_client: Setting Primary Keys constraint ['id_cliente'] on table 'databricks_clase.prueba_schema.base_consolidada_mensual_feats_v2'.
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:60: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:2570: SyntaxWarning: invalid escape sequence '\.'
  """The sequence number of this run attempt for a triggered job run. The initial attempt of a run
/databricks/python/lib/python3.12/site-packages/databricks/sdk/service/jobs.py:3431: SyntaxWarning: invalid escape sequence '\.'
  """The se

In [0]:
# --- 2.2. Tabla 2: Features históricos (simulando otro set de características derivadas) ---
# En este ejemplo, calculamos dos features adicionales a partir de la data mensual
df_historico = df_mensual.withColumn("avg_monto", expr("(monto_1m + monto_2m + monto_3m) / 3")) \
                         .withColumn("max_monto", greatest(col("monto_1m"), col("monto_2m"), col("monto_3m")))

feature_columns_2 = ["id_cliente", "avg_monto", "max_monto"]
feature_table_name_2 = "databricks_clase.prueba_schema.historico_feats"
df_features_2 = df_historico.select(*feature_columns_2).dropDuplicates(["id_cliente"])

# Crear la tabla de features 2 en el Feature Store
fs.create_table(
    name=feature_table_name_2,
    primary_keys=["id_cliente"],
    schema=df_historico.select(*feature_columns_2).schema,
    description="Feature Store con features históricos derivados (promedio y máximo de monto)"
)

# Escribir la data en la tabla de features históricos

fs.write_table(
    name=feature_table_name_2,
    df=df_features_2,
    mode="overwrite"
)

In [0]:
# -----------------------------------------------------------
# 3. Unir las dos tablas de Feature Store y obtener la data para entrenamiento/inferencia
# -----------------------------------------------------------
# Lectura de las dos tablas de Feature Store
features_df1 = fs.read_table(name=feature_table_name_1)
features_df2 = fs.read_table(name=feature_table_name_2)

# Realizamos el join usando la clave primaria 'id_cliente'
features_joined = features_df1.join(features_df2, on="id_cliente", how="inner")

# Para obtener el target, unimos la data de features con la tabla consolidada original
target_df = spark.table("databricks_clase.prueba_schema.base_consolidada_v2").select("id_cliente", "flg_churn")
features_final = features_joined.join(target_df, on="id_cliente", how="inner")

# Convertimos a Pandas para usar con scikit-learn
features_pd = features_final.toPandas()

# Definimos las features (columnas) y el target para el modelo
feature_list = [
    'frecuencia_1m', 'cantidad_1m', 'tasa', 'ultima_compra_1m', 'monto_total', 
    'avg_monto', 'max_monto'
]
X = features_pd[feature_list]
y = features_pd["flg_churn"]

In [0]:
# -----------------------------------------------------------
# 4. Entrenamiento simple del modelo
# -----------------------------------------------------------
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, classification_report

# Dividimos la data en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

with mlflow.start_run():
    # Creamos y entrenamos un modelo simple (Random Forest)
    model_train = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, class_weight="balanced")
    model_train.fit(X_train, y_train)
    
    # Realizamos predicciones en el conjunto de prueba
    y_pred_train = model_train.predict(X_test)
    
    signature = infer_signature(X_test, y_test)  # Inferimos la firma con datos de prueba y predicciones

    # Calculamos métricas de desempeño
    f1 = f1_score(y_test, y_pred_train)
    accuracy = accuracy_score(y_test, y_pred_train)
    print(f"F1 Score (entrenamiento): {f1:.4f}")
    print(f"Accuracy (entrenamiento): {accuracy:.4f}")
    print("Reporte de clasificación (entrenamiento):\n", classification_report(y_test, y_pred_train))
    
    # Registramos parámetros y el modelo en MLflow
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 10)
    mlflow.log_metric("f1_score", f1)
    mlflow.log_metric("accuracy", accuracy)
    #mlflow.sklearn.log_model(model_train, "modelo_simple")

    mlflow.sklearn.log_model(
        sk_model=model_train,
        artifact_path="modelo_simple",
        signature=signature,  # Agregamos la firma aquí
        input_example=X_test.iloc[:5]  # Opcional pero recomendable para referencia
    )
    # Opcional: registrar el modelo en el Model Registry
mlflow.end_run()

F1 Score (entrenamiento): 0.4418
Accuracy (entrenamiento): 0.7717
Reporte de clasificación (entrenamiento):
               precision    recall  f1-score   support

         0.0       0.96      0.78      0.86     68203
         1.0       0.31      0.74      0.44      9491

    accuracy                           0.77     77694
   macro avg       0.64      0.76      0.65     77694
weighted avg       0.88      0.77      0.81     77694



Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

2025/02/26 02:14:33 INFO mlflow.tracking._tracking_service.client: 🏃 View run dapper-wolf-739 at: adb-106485471189205.5.azuredatabricks.net/ml/experiments/d6aca2d1639b4b1498475591096c2c2c/runs/8be5abd7b4254acbb00041830e627f47.
2025/02/26 02:14:33 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: adb-106485471189205.5.azuredatabricks.net/ml/experiments/d6aca2d1639b4b1498475591096c2c2c.
